## Item 44: Use Plain Attributes Instead of Setter and Getter Methods

In [1]:
class Resistor:
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0

r1 = Resistor(50e3)
r1.ohms = 10e2
print(r1.ohms)

1000.0


In [2]:
#@property를 이용한 getterrhk setter 사용

class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0
    
    @property
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms

r2 = VoltageResistance(1e3)
print(f'Before: {r2.current:.2f} amps')
r2.voltage = 10 #setter 작동
print(f'After: {r2.current:.2f} amps')

Before: 0.00 amps
After: 0.01 amps


## Item 45: Consider @property Instead of Reinfactoring Attributes
1. Use @property to give existing instance attributes new functionality.
2. Make incremental progress toward better data models by using @property

## Item 46: Use Descriptors for Reusable @property Methods

 Reuse the behavior and validation of @property methods by defining your own descriptor classes


In [3]:
class Grade: #descriptor class
    def __init__(self):
        self._value = 0
    
    def __get__(self, instance, instance_type): # self는 Grade class의 인스턴스를 뜻함. instance는 Exam class의 인스턴스를 뜻함.
        return self._value
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError(
                'Grade must be between 0 and 100')
        self._value = value

class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)

print('problem....')
second_exam = Exam()
second_exam.writing_grade = 75
print('Writing', first_exam.writing_grade) #문제발생, 인스턴스가 다른데 사용하는 디스크립터 객체가 같기 때문
print('Writing', second_exam.writing_grade)

Writing 82
Science 99
problem....
Writing 75
Writing 75


In [4]:
class Grade: #descriptor class
    def __init__(self):
        self._values = {}
    
    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return self._values.get(instance, 0)
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError(
                'Grade must be between 0 and 100')
        self._values[instance] = value

class Exam: #owner class
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

print('problem....')
second_exam = Exam()
second_exam.writing_grade = 75
print('Writing', first_exam.writing_grade) #문제해결
print('Writing', second_exam.writing_grade)

problem....
Writing 75
Writing 75


## Item 47: Use __getattr__, __gatattribute__, and __setattr__ for Lazy Attributes

In [5]:
class LazyRecord:
    def __init__(self):
        self.exists = 5
    
    def __getattr__(self, name): #name에 해당하는 attribute가 없는데 호출될 경우 작동
        value = f'Value for {name}'
        setattr(self, name, value)
        return value

data = LazyRecord()
print('Before:', data.__dict__)
print('foo:   ', data.foo)
print('After: ', data.__dict__)

Before: {'exists': 5}
foo:    Value for foo
After:  {'exists': 5, 'foo': 'Value for foo'}


In [6]:
class ValidatingRecord:
    def __init__(self):
        self.exists = 5
    
    def __getattribute__(self, name): #name에 해당하는 attribute가 호출될 경우 작동
        try:
            value = super().__getattribute__(name)
            print(f'found {name!r}, returning {value!r}')
            return value
        except AttributeError:
            value = f'Value for {name}'
            print(f'Setting {name!r} to {value!r}')
            setattr(self, name, value)
            return value

    
data = ValidatingRecord()
print('Before:', data.exists)
print('foo:   ', data.foo)
print('After: ', data.foo)

found 'exists', returning 5
Before: 5
Setting 'foo' to 'Value for foo'
foo:    Value for foo
found 'foo', returning 'Value for foo'
After:  Value for foo


## Item 48: Validate Subclasses with __init_subclass___

In [7]:
class Meta(type): #mataclass, type을 반드시 상속
    def __new__(meta, name, bases, class_dict): #__new__는 class가 선언될때 작동하는 메서드임. __init__은 class의 인스턴스 선언될때 작동함
        print(f'Running {meta}.__new__ for {name}') 
        print('Bases:', bases) #부모클레스가 무엇인지 접근
        print(class_dict) #class의 attribute가 뭐가 있는지 접근(함수포함)
        return type.__new__(meta, name, bases, class_dict)

class MyClass(metaclass=Meta):
    def __init__(self):
        print('__init__')
    
    def foo(self):
        pass

class MySubclass(MyClass):
    def bar(self):
        pass

Running <class '__main__.Meta'>.__new__ for MyClass
Bases: ()
{'__module__': '__main__', '__qualname__': 'MyClass', '__init__': <function MyClass.__init__ at 0x0000010F81C05D30>, 'foo': <function MyClass.foo at 0x0000010F81C05EE0>}
Running <class '__main__.Meta'>.__new__ for MySubclass
Bases: (<class '__main__.MyClass'>,)
{'__module__': '__main__', '__qualname__': 'MySubclass', 'bar': <function MySubclass.bar at 0x0000010F81C5A9D0>}


In [8]:
class Filled:
    color = None # Must be specified by subclasses

    def __init_subclass__(cls): #classmethod
        super().__init_subclass__()
        if cls.color not in ('red', 'green', 'blue'):
            raise ValueError('Fills need a valid color')

class RedTriangle(Filled):
    # color = 'pupple' -> ValueError 발생
    color = 'red'
    sides =3

## Item 49: Register Class Existence with __init_subclass__

In [9]:
import json

class BetterSerializable:
    def __init__(self, *args):
        self.args = args
    
    def serialize(self):
        return json.dumps({
            'class' : self.__class__.__name__,
            'args': self.args,
        })
    
    def __repr__(self):
        name = self.__class__.__name__
        args_str = ', '.join(str(x) for x in self.args)
        return f'{name}({args_str})'


registry = {}
def register_class(target_class):
    registry[target_class.__name__] = target_class

def deserialize(data):
    params = json.loads(data)
    name = params['class']
    target_class = registry[name]
    return target_class(*params['args'])

In [10]:
class Meta(type):#meta class를 활용하여, class registration
    def __new__(meta, name, bases, class_dict):
        cls = type.__new__(meta, name, bases, class_dict)
        register_class(cls)
        return cls

class RegisteredSerializable(BetterSerializable, metaclass=Meta):
    pass

class Vector3d(RegisteredSerializable):
    def __init__(self, x, y, z):
        super().__init__(x, y, z)
        self.x, self.y, self.z = x, y, z

print(registry)
before = Vector3d(10, -7, 3)
print('Before:    ', before)
data = before.serialize()
print('Serialized:', data)
print('After:     ', deserialize(data))

{'RegisteredSerializable': <class '__main__.RegisteredSerializable'>, 'Vector3d': <class '__main__.Vector3d'>}
Before:     Vector3d(10, -7, 3)
Serialized: {"class": "Vector3d", "args": [10, -7, 3]}
After:      Vector3d(10, -7, 3)


In [11]:
registry = {}

class BetterRegisteredSerializable(BetterSerializable):
    def __init_subclass__(cls): #class registration을 위해 사용, metaclass보다 가시성이 좋음
        super().__init_subclass__()
        register_class(cls)

class Vector1D(BetterRegisteredSerializable):
    def __init__(self, magnitude):
        super().__init__(magnitude)
        self.magnitude = magnitude

print(registry)
before = Vector1D(6)
print('Before:    ', before)
data = before.serialize()
print('Serialized:', data)
print('After:     ', deserialize(data))

{'Vector1D': <class '__main__.Vector1D'>}
Before:     Vector1D(6)
Serialized: {"class": "Vector1D", "args": [6]}
After:      Vector1D(6)


## Item 50: Annotate Class Attributes with __set_name__

In [12]:
class Field:
    def __init__(self):
        self.name = None
        self.internal_name = None
    
    def __set_name__(self, owner, name):#여기서 owner는 containing class를 뜻함.
        self.name = name
        self.internal_name = '_' + name
    
    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return getattr(instance, self.internal_name, '')
    
    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)

class FixedCustomer:
    first_name = Field() #__set_name__실행됨.
    last_name = Field()
    prefix = Field()
    suffix = Field()

cust = FixedCustomer()
print(f'Before: {cust.first_name!r} {cust.__dict__}')
cust.first_name = 'Mersenne'
print(f'After: {cust.first_name!r} {cust.__dict__}')


Before: '' {}
After: 'Mersenne' {'_first_name': 'Mersenne'}


## Item 51: Prefer Class Decorators Over Metaclasses for Composable Class Extensions

In [13]:
#class decorator

def my_class_decorator(klass):
    klass.extra_param = 'hello'
    return klass

@my_class_decorator #my_class_decorator(Myclass)라 생각하면 됨.
class Myclass:
    pass

print(Myclass)
print(Myclass.extra_param)

<class '__main__.Myclass'>
hello


In [14]:
from functools import wraps
import types

def trace_func(func):
    if hasattr(func, 'tracing'):
        return func
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = None
        try:
            result = func(*args, **kwargs)
            return result
        except Exception as e:
            result = e
            raise
        finally:
            print(f'{func.__name__}({args!r}, {kwargs!r}) -> '
                  f'{result!r}')

    wrapper.tracing = True
    return wrapper

trace_types = (
    types.MethodType,
    types.FunctionType,
    types.BuiltinFunctionType,
    types.BuiltinMethodType,
    types.MethodDescriptorType,
    types.ClassMethodDescriptorType)


In [15]:
def trace(klass):
    for key in dir(klass):
        value = getattr(klass, key)
        if isinstance(value, trace_types):
            wrapped = trace_func(value)
            setattr(klass, key, wrapped)
    return klass

@trace
class TraceDict(dict):
    pass

trace_dict = TraceDict([('hi', 1)])
trace_dict['there'] = 2
trace_dict['hi']
try:
    trace_dict['does not exist']
except KeyError:
    pass

__new__((<class '__main__.TraceDict'>, [('hi', 1)]), {}) -> {}
__getitem__(({'hi': 1, 'there': 2}, 'hi'), {}) -> 1
__getitem__(({'hi': 1, 'there': 2}, 'does not exist'), {}) -> KeyError('does not exist')
